In [3]:
import pandas as pd
import os
import numpy as np

In [4]:
# Able to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

orig_SR = 44100
orig_blocksize = int(orig_SR * 5)
orig_overlap = 0 #int(orig_SR/4)

SR = 16000
blocksize = int(SR * 5)
overlap = 0 #int(SR/4)

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load Dataset
Here we load the csv that describes each file in the dataset. We add a high level category that is defined in the ESC-50 documentation. This we realize is anthetical to true training, it is a stopgap for when we use NLP to classify tags into these categories.

In [6]:
path_to_db='../../ESC-50/'
audio_dir = path_to_db + 'audio/'
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {'dog': 0,'rooster': 0,'pig': 0,'cow': 0,'frog': 0,'cat': 0,'hen': 0,
            'insects': 0,'sheep': 0,'crow': 0,'rain': 1,'sea_waves': 1,'crackling_fire': 1,
            'crickets': 0,'chirping_birds': 0,'water_drops': 1,'wind': 1,'pouring_water': 1,
            'toilet_flush': 1,'thunderstorm': 1,'crying_baby': 0,'sneezing': 0,'clapping': 0,
            'breathing': 0,'coughing': 0,'footsteps': 1,'laughing': 0,'brushing_teeth': 1,
            'snoring': 0,'drinking_sipping': 1,'door_wood_knock': 1,'mouse_click': 1,
            'keyboard_typing': 1,'door_wood_creaks': 1,'can_opening': 1,'washing_machine': 1,
            'vacuum_cleaner': 1,'clock_alarm': 1,'clock_tick': 1,'glass_breaking':1,'helicopter': 1,
            'chainsaw': 1,'siren': 1,'car_horn': 1,'engine': 1,'train': 1,'church_bells': 1,
            'airplane': 1,'fireworks': 1,'hand_saw': 1,
            }
dataset['h_target'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_target'] = mapping[row['category']]

In [7]:
dataset.head()

,filename,fold,target,category,esc10,src_file,take,h_target
0,1-100032-A-0.wav,1,0,dog,True,100032,A,0
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,0
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,1
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,1
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,1


In [10]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self,):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 9)
        self.conv2 = nn.Conv2d(32, 32, 5)
        self.norm = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(7, 7)
        self.fc1 = nn.Linear(256, 64)
        self.dropout = nn.Dropout2d(p=0.2)
#         self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.norm(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(9, 9), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout2d(p=0.2, inplace=False)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (softmax): LogSoftmax()
)


In [9]:
# load data:
from PIL import Image
import os 
import numpy as np
from torch.utils.data import Dataset, DataLoader

class trainset(Dataset):
    def __init__(self):
        
        root = "./data2/"
        self.data_list = []
        self.label_list = []
        for root, dir, files in os.walk("./data2/"):
            for file in files:
                if file.find('.npy')!= -1:
                    self.data_list.append(os.path.join(root, file))
                    self.label_list.append(int(root[13:]))
        print(len(self.data_list), len(self.label_list))
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):

        path = self.data_list[index]
#         data = []
#         label = []
        fname = path.split("/")[-1][:-4]+".wav"
#         print(fname)
        data = np.load(path,allow_pickle = True)
        data = np.expand_dims(data, axis=0)
#         data = np.asarray(Image.open(path))
        label = float(dataset[dataset["filename"]==fname]["h_target"].values[0])
#         print(label)
    
#         print(data.shape)
#         data = np.asarray(data)
        label = np.asarray(label)
        
        return data, label
dataloader = trainset()
data,label = dataloader.__getitem__(5)
# print(label.shape)

2000 2000


In [19]:
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import time

# model = BaseNet()
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

validation_split = .2
random_seed= 42
shuffle_dataset = True
dataset_size = 2000
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

datasets  = trainset()
trainloader = torch.utils.data.DataLoader(datasets, batch_size=128, 
                                           sampler=train_sampler)
valloader = torch.utils.data.DataLoader(datasets, batch_size=128,
                                                sampler=valid_sampler)

val_history = []
val_loss_hist = []
train_history = []
train_loss_hist = []

for epoch in range(30):  # loop over the dataset multiple times
    
########## Validation ###########
    
    count = 0
    running_accuracy = 0
    running_loss = 0
    t1 = time.time()
    for i, data in enumerate(valloader, 0):
        count += 1
        inputs, labels = data
        labels = labels.long()
        inputs = inputs.float()
        outputs = model(inputs)
        val_loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        acc_val = torch.eq(preds, labels).float().mean()
        running_accuracy += acc_val.item()
        running_loss += val_loss.item()
        
    running_accuracy /= count
    running_loss /= count
    val_history.append(running_accuracy)
    val_loss_hist.append(running_loss)
    t2 = time.time()
    print("===========Phase: Val============")
    print("Validation Time: {}".format(t2 - t1))
    print("Epoch: {}  val_loss: {}".format(epoch, running_loss))
    print("Epoch: {}  val_accuracy: {}".format(epoch, running_accuracy))
    
######### Training ###########   
    
    running_loss = 0.0
    count = 0
    training_accuracy = 0
    t1 = time.time()
    for i, data in enumerate(trainloader, 0):
        count += 1
        inputs, labels = data
        inputs = inputs.float()
        labels = labels.long()
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        acc_train = torch.eq(preds, labels).float().mean()
        running_accuracy += acc_train.item()
        
    running_accuracy /= count
    running_loss /= count
    train_history.append(running_accuracy)
    train_loss_hist.append(running_loss)
    t2 = time.time()
    print("===========Phase: Train============") 
    print("Training Time: {}".format(t2 - t1))
    print("Epoch: {}  train_loss: {}".format(epoch, running_loss))
    print("Epoch: {}  train_accuracy: {}".format(epoch, running_accuracy))
    print()
        
# # #             torch.save(net, 'toy_model.pt')
        
    
print('Finished Training')


2000 2000
===========Phase: Val============
Validation Time: 14.660237550735474
Epoch: 0  val_loss: 0.6985870897769928
Epoch: 0  val_accuracy: 0.462890625
===========Phase: Train============
Training Time: 171.8205370903015
Epoch: 0  train_loss: 0.6489567664953378
Epoch: 0  train_accuracy: 0.6359675480769231

===========Phase: Val============
Validation Time: 14.037136793136597
Epoch: 1  val_loss: 0.6450876593589783
Epoch: 1  val_accuracy: 0.615234375
===========Phase: Train============
Training Time: 167.29679131507874
Epoch: 1  train_loss: 0.6152861072466924
Epoch: 1  train_accuracy: 0.6891526442307693

===========Phase: Val============
Validation Time: 14.475305795669556
Epoch: 2  val_loss: 0.6146249771118164
Epoch: 2  val_accuracy: 0.66796875
===========Phase: Train============
Training Time: 168.85869979858398
Epoch: 2  train_loss: 0.5798344749670762
Epoch: 2  train_accuracy: 0.7286658653846154

===========Phase: Val============
Validation Time: 14.698755979537964
Epoch: 3  val_lo

===========Phase: Val============
Validation Time: 14.040440320968628
Epoch: 27  val_loss: 0.3476894944906235
Epoch: 27  val_accuracy: 0.83984375
===========Phase: Train============
Training Time: 168.33998727798462
Epoch: 27  train_loss: 0.25799522262353164
Epoch: 27  train_accuracy: 0.9660456730769231

===========Phase: Val============
Validation Time: 14.196431159973145
Epoch: 28  val_loss: 0.39139188826084137
Epoch: 28  val_accuracy: 0.853515625
===========Phase: Train============
Training Time: 168.49377155303955
Epoch: 28  train_loss: 0.2850327342748642
Epoch: 28  train_accuracy: 0.9532752403846154

===========Phase: Val============
Validation Time: 14.146916151046753
Epoch: 29  val_loss: 0.39415503293275833
Epoch: 29  val_accuracy: 0.8125
===========Phase: Train============
Training Time: 168.4154772758484
Epoch: 29  train_loss: 0.24700509126369768
Epoch: 29  train_accuracy: 0.9645432692307693

Finished Training


In [21]:
torch.save(model, 'hierarchical_cnn.pt')

## High-Level Shallow Nets
Train binary shallow nets for high level categories(animals, natural, human, domestic, urban)

In [13]:
from cnnmult import CNN_Multilayer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.decomposition import PCA

clf = CNN_Multilayer()

clf.fit(train_X, train_y)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_1 (Melspectro (None, 128, 313, 1)       296064    
_________________________________________________________________
conv1 (Conv2D)               (None, 120, 305, 32)      2624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 17, 32)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2176)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                69664     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________

942/942 [==============================] - 26s 27ms/step - loss: 3.1531 - acc: 0.0817 - val_loss: 3.2304 - val_acc: 0.0200
Epoch 8/50
942/942 [==============================] - 27s 29ms/step - loss: 3.1282 - acc: 0.0828 - val_loss: 3.2173 - val_acc: 0.0200
Epoch 9/50
942/942 [==============================] - 27s 28ms/step - loss: 3.0945 - acc: 0.1136 - val_loss: 3.1836 - val_acc: 0.0200
Epoch 10/50
942/942 [==============================] - 27s 28ms/step - loss: 3.0966 - acc: 0.1178 - val_loss: 3.1231 - val_acc: 0.0400
Epoch 11/50
942/942 [==============================] - 26s 28ms/step - loss: 3.0418 - acc: 0.1316 - val_loss: 3.1072 - val_acc: 0.0200
Epoch 12/50
942/942 [==============================] - 26s 28ms/step - loss: 3.0123 - acc: 0.1338 - val_loss: 3.1074 - val_acc: 0.0800
Epoch 13/50
942/942 [==============================] - 27s 28ms/step - loss: 2.9774 - acc: 0.1497 - val_loss: 3.0710 - val_acc: 0.0800
Epoch 14/50
942/942 [==============================] - 26s 28ms/step 

CNN_Multilayer(a_batch_size=128, a_epochs=50, batch_size=128, epochs=50,
        i_batch_size=128, i_epochs=50, validation_split=0.05, verbose=1)